In [6]:
# 1. 파일 불러오기
import pandas as pd

temp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균기온정규화.csv",encoding='euc-kr')
temp_norm.index
temp_norm = temp_norm.set_index("district")

rain_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균강수량정규화.csv",encoding='euc-kr')
rain_norm.index
rain_norm = rain_norm.set_index("district")

dust_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//대기오염정규화.csv",encoding='euc-kr')
dust_norm.index
dust_norm = dust_norm.set_index("district")

flt_pp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//유동인구정규화.csv",encoding='euc-kr')
flt_pp_norm.index
flt_pp_norm = flt_pp_norm.set_index("district")

park_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//관광지.csv",encoding='euc-kr')
        
rest_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//음식점.csv",encoding='euc-kr')

In [7]:
# 2. 제주도 읍,면,동별 경계선 데이터 불러오기
import json
geo_path = 'C://Users//whs38//캡스톤//jeju_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

{'type': 'FeatureCollection',
 'name': 'jeju_map',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'EMD_CD': '50130259',
    'EMD_NM': '성산읍',
    'SGG_OID': 2322,
    'COL_ADM_SE': '50130',
    'GID': 5786},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[126.80030580596758, 33.38207390893856],
       [126.80038962856278, 33.38237476052503],
       [126.80103043573062, 33.38280017778068],
       [126.80140861980678, 33.38347945155027],
       [126.80199687144263, 33.38385941882818],
       [126.80211081536473, 33.38387757999417],
       [126.80256298908803, 33.38396531990157],
       [126.80326089609893, 33.38407855155361],
       [126.80347241134024, 33.384107875873795],
       [126.80365484724061, 33.38414076899815],
       [126.80448723612078, 33.384469459536675],
       [126.80526223466302, 33.38484376573159],
       [126.8056956543542, 33.38504065923038],
       [126.80605413675

In [8]:
 # 3. 가중치 부여하기 
temp_norm = temp_norm*1      #int(RadioVariety.get())
rain_norm = rain_norm*1      #int(RadioVariety2.get())
dust_norm = dust_norm*1      #int(RadioVariety3.get())
flt_pp_norm = flt_pp_norm*1  #int(RadioVariety4.get())

In [9]:
# 4. 지도에 시각화 하기 
import folium
total_map = folium.Map(location=[33.362500, 126.533694], zoom_start=11,
                           tiles='Stamen Terrain') 

total_map.choropleth(geo_data = geo_str,
                    data = temp_norm["6월"],   #  combobox.get()  
                    columns = [temp_norm.index, temp_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "기온")

        
total_map.choropleth(geo_data = geo_str,
                    data = rain_norm["6월"],   #  combobox.get()  
                    columns = [rain_norm.index, rain_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "강수량")
        
total_map.choropleth(geo_data = geo_str,
                    data = dust_norm["6월"],   #  combobox.get()  
                    columns = [dust_norm.index, dust_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "대기오염도")
        
total_map.choropleth(geo_data = geo_str,
                    data = flt_pp_norm["6월"],   #  combobox.get()  
                    columns = [flt_pp_norm.index, flt_pp_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "유동인구")

 # 관광지 MarkerCluster
from folium.plugins import MarkerCluster

p_lat = park_loc.lat.values
p_lng = park_loc.lng.values
p_locations = list(zip(p_lat,p_lng))
p_popups = list(park_loc.name.values)
p_icons = [folium.Icon(icon="tree", prefix="fa", color="green") for _ in range(len(p_locations))]

p_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-small',
    iconSize: new L.Point(30, 30)
    });
}"""

p_marker_cluster = MarkerCluster(
    locations=p_locations, popups=p_popups,
    name='공원',
    overlay=True,
    control=True,
    icon_create_function = p_icon_create_function,
    icons = p_icons)


p_marker_cluster.add_to(total_map)

# 음식점 위치
r_lat = rest_loc.lat.values
r_lng = rest_loc.lng.values
r_locations = list(zip(r_lat,r_lng))
r_popups = list(rest_loc.restaurant.values)
r_icons = [folium.Icon(icon="heart", prefix="fa", color="orange") for _ in range(len(r_locations))]

r_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""

r_marker_cluster = MarkerCluster(
    locations=r_locations, popups=r_popups,
    name='맛집',
    overlay=True,
    control=True,
    icon_create_function = r_icon_create_function,
    icons = r_icons)


r_marker_cluster.add_to(total_map)


In [10]:
folium.LayerControl().add_to(total_map)

save_path = "C://Users//whs38//캡스톤//시각화" + "nooverlap" + ".html"    #str(combobox.get())
total_map.save(save_path)

In [ ]:
# 1. 파일 불러오기
import pandas as pd

temp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균기온정규화.csv",encoding='euc-kr')
temp_norm.index
temp_norm = temp_norm.set_index("district")

rain_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//평균강수량정규화.csv",encoding='euc-kr')
rain_norm.index
rain_norm = rain_norm.set_index("district")

dust_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//대기오염정규화.csv",encoding='euc-kr')
dust_norm.index
dust_norm = dust_norm.set_index("district")

flt_pp_norm = pd.read_csv("C://Users//whs38//캡스톤//데이터//유동인구정규화.csv",encoding='euc-kr')
flt_pp_norm.index
flt_pp_norm = flt_pp_norm.set_index("district")

park_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//관광지.csv",encoding='euc-kr')
        
rest_loc = pd.read_csv("C://Users//whs38//캡스톤//데이터//음식점.csv",encoding='euc-kr')

# 2. 제주도 읍,면,동별 경계선 데이터 불러오기
import json
geo_path = 'C://Users//whs38//캡스톤//jeju_map.geojson'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

 # 3. 가중치 부여하기 
temp_norm = temp_norm*1      #int(RadioVariety.get())
rain_norm = rain_norm*1      #int(RadioVariety2.get())
dust_norm = dust_norm*1      #int(RadioVariety3.get())
flt_pp_norm = flt_pp_norm*1  #int(RadioVariety4.get())

# 4. 지도에 시각화 하기 
import folium
total_map = folium.Map(location=[33.362500, 126.533694], zoom_start=11,
                           tiles='Stamen Terrain') 

total_map.choropleth(geo_data = geo_str,
                    data = temp_norm["6월"],   #  combobox.get()  
                    columns = [temp_norm.index, temp_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "기온")

        
total_map.choropleth(geo_data = geo_str,
                    data = rain_norm["6월"],   #  combobox.get()  
                    columns = [rain_norm.index, rain_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "강수량")
        
total_map.choropleth(geo_data = geo_str,
                    data = dust_norm["6월"],   #  combobox.get()  
                    columns = [dust_norm.index, dust_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "대기오염도")
        
total_map.choropleth(geo_data = geo_str,
                    data = flt_pp_norm["6월"],   #  combobox.get()  
                    columns = [flt_pp_norm.index, flt_pp_norm["6월"]], # combobox.get()
                    fill_color = 'PuRd', #PuRd, YlGnBu
                    key_on = 'feature.properties.EMD_NM',
                    name = "유동인구")

 # 공원 MarkerCluster
from folium.plugins import MarkerCluster

p_lat = park_loc.lat.values
p_lng = park_loc.lng.values
p_locations = list(zip(p_lat,p_lng))
p_popups = list(park_loc.name.values)
p_icons = [folium.Icon(icon="tree", prefix="fa", color="green") for _ in range(len(p_locations))]

p_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-small',
    iconSize: new L.Point(30, 30)
    });
}"""

p_marker_cluster = MarkerCluster(
    locations=p_locations, popups=p_popups,
    name='공원',
    overlay=True,
    control=True,
    icon_create_function = p_icon_create_function,
    icons = p_icons)


p_marker_cluster.add_to(total_map)

# 맛집 위치
r_lat = rest_loc.lat.values
r_lng = rest_loc.lng.values
r_locations = list(zip(r_lat,r_lng))
r_popups = list(rest_loc.restaurant.values)
r_icons = [folium.Icon(icon="heart", prefix="fa", color="orange") for _ in range(len(r_locations))]

r_icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    });
}"""

r_marker_cluster = MarkerCluster(
    locations=r_locations, popups=r_popups,
    name='맛집',
    overlay=True,
    control=True,
    icon_create_function = r_icon_create_function,
    icons = r_icons)


r_marker_cluster.add_to(total_map)

folium.LayerControl().add_to(total_map)

save_path = "C://Users//whs38//캡스톤//시각화" + "nooverlap" + ".html"    #str(combobox.get())
total_map.save(save_path)